# Udtræk tekst fra Lokalplaner
Henter tekst ud af PDF'er fra plansystem og lægger det i en PostgreSQL tabel.

In [1]:
import requests
import io
import PyPDF2
import logging
import psycopg2
from sqlalchemy import create_engine, Table, MetaData, update, select

In [2]:
import sys
sys.path.append('/python/')
import connections as con

https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [8]:
def get_document(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e.message, e.args)
        logging.exception(e.message)
    pdf_file = io.BytesIO(r.content)
    pdfReader = PyPDF2.PdfFileReader(pdf_file)

    #discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    #The while loop will read each page
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
# Nedenfor bruges OCR til scanning af dokumenter der ikke er maskinlæsbare. Skal lige kigges igennem.
#    if text != "":
#       text = text
#    #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
#    else:
#       text = textract.process(pdf_file, method='tesseract', language='eng')
    return text

In [9]:
engine = con.engine('production')

In [10]:
connection = engine.connect()

In [11]:
metadata = MetaData()

In [9]:
plan = Table('lokalplan_dokument', metadata, autoload=True, autoload_with=engine, schema='plandatadk')

In [10]:
def update_table(komkode):
    sql = '''
        select PLANID, STATUS, doklink
        FROM plandatadk.lokalplan_mv
        where STATUS in ('V', 'F') and AKTUEL = true and komnr = {} and doklink is not null
        '''.format(komkode)
    result_set = connection.execute(sql)  
    for r in result_set:  
        plan_update = plan.update().values(document=get_document(r['doklink'])).where(plan.c.planid == r['planid'] and plan.c.status == r['status'])
        engine.execute(plan_update)
        logging.info('Indlæst: %s', plan.c.planid)

In [11]:
if __name__ == "__main__":
    update_table(147)

# Gammelt

In [ ]:
class Plandokument():
    def get_document(self, url):
        try:
            r = requests.get(url)
        except Exception as e:
            print(e.message, e.args)
        pdf_file = io.BytesIO(r.content)
        pdfReader = PyPDF2.PdfFileReader(pdf_file)

        #discerning the number of pages will allow us to parse through all #the pages
        num_pages = pdfReader.numPages
        count = 0
        text = ""
        #The while loop will read each page
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()

        return text
    
    def connect_to_db(self, user, pw, host, port, db):
        engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(self.user, self.pw, self.host, self.port, self.db))
        
    

In [ ]:
import logging
logging.info('Test')

In [ ]:
plan = Table('lokalplan_dokument', metadata, autoload=True, autoload_with=engine)

In [ ]:
fileurl = 'pdf/noocr.pdf'
text = textract.process(fileurl, method='tesseract', language='eng')

In [ ]:
print(text)